# Preprocessing

In [9]:
import pandas as pd

In [10]:
df = pd.read_csv('adult.data', header=None)

In [11]:
# Initial data stats
df.shape

(32561, 15)

## 1. Data Cleaning

In [13]:
# rename columns
columns = ['age', 'workclass', 'final_weight', 'education', 'education_num',
           'marital_status', 'occupation', 'relationship', 'race',
           'sex', 'capital_gain', 'capital_loss', 'hours_per_week',
           'native_country', 'income']
df.columns=columns

In [14]:
# Continuous vs. Discrete independent variables
columns_cont = ['age', 'capital_gain', 'capital_loss', 'hours_per_week']
columns_dis = ['workclass', 'education_num', 'occupation',
               'race', 'sex', 'native_country']

In [15]:
# Remove leading white space in some columns
to_strip = ['workclass', 'education', 'marital_status', 'occupation',
            'race', 'sex', 'native_country', 'income']

df[to_strip] = df[to_strip].apply(lambda x: x.str.strip())

### Drop duplicates

In [16]:
# drop 24 duplicate rows
df.drop_duplicates(keep='first', inplace=True)

In [17]:
df.shape

(32537, 15)

### Drop ambiguous data values

In [18]:
# To Drop - uncertain value
uncertain_marital = ['Never-married', 'Widowed', 'Married-AF-spouse']
uncertain_country = ['?', 'South']
uncertain_occupation = ['?']

In [19]:
# Drop rows where marital_status is not relevant (11,699 rows)
df = df[~df.marital_status.isin(uncertain_marital)]

In [20]:
# Drop rows where native_country is not relevant (423 rows)
df = df[~df.native_country.isin(uncertain_country)]

In [21]:
# Drop rows where occupation is not relevant (897 rows)
df = df[~df.occupation.isin(uncertain_occupation)]

### Drop irrelevant data features

In [23]:
# Remove 2 columns from original dataset that are not used
df.drop(['final_weight', 'relationship'], axis=1, inplace=True)

## 2. Data regrouping/relabeling

### Income

In [24]:
# Relabel income column to adjust for inflation
df.replace({'<=50K': '<=85k', '>50K':'>85k'}, inplace=True)

In [25]:
df.income.value_counts()

<=85k    12602
>85k      6932
Name: income, dtype: int64

### Education

In [26]:
# Check if "education" and "education_num" are the same
df.education.value_counts()

HS-grad         6574
Some-college    3972
Bachelors       3240
Masters         1204
Assoc-voc        934
Assoc-acdm       678
11th             504
10th             487
Prof-school      448
7th-8th          408
Doctorate        301
9th              301
5th-6th          198
12th             166
1st-4th           98
Preschool         21
Name: education, dtype: int64

In [23]:
df.education_num.value_counts()

9     6574
10    3972
13    3240
14    1204
11     934
12     678
7      504
6      487
15     448
4      408
5      301
16     301
3      198
8      166
2       98
1       21
Name: education_num, dtype: int64

In [28]:
# Group together all who did not graduate high school
educ_num_map = {i:8 for i in range(1,8)}

educ_map = {
    'Preschool': '<HS',
    '1st-4th': '<HS',
    '5th-6th': '<HS',
    '7th-8th': '<HS',
    '9th': '<HS',
    '10th': '<HS',
    '11th': '<HS',
    '12th': '<HS'
}

df.replace({'education_num': educ_num_map}, inplace=True)
df.replace({'education': educ_map}, inplace=True)

In [29]:
# Check if map was accurate
df.education.value_counts()

HS-grad         6574
Some-college    3972
Bachelors       3240
<HS             2183
Masters         1204
Assoc-voc        934
Assoc-acdm       678
Prof-school      448
Doctorate        301
Name: education, dtype: int64

### Marital Status (dependent variable)

In [30]:
# Negative marriage outcome
neg_marital_status = ['Divorced', 'Separated', 'Married-spouse-absent']

# Positive marriage outcome
married = df.marital_status == 'Married-civ-spouse'
not_married = df.marital_status.isin(neg_marital_status)

In [31]:
df[not_married]

,age,workclass,education,education_num,marital_status,occupation,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,White,Male,0,0,40,United-States,<=85k
6,49,Private,<HS,8,Married-spouse-absent,Other-service,Black,Female,0,0,16,Jamaica,<=85k
19,43,Self-emp-not-inc,Masters,14,Divorced,Exec-managerial,White,Female,0,0,45,United-States,>85k
21,54,Private,HS-grad,9,Separated,Other-service,Black,Female,0,0,20,United-States,<=85k
24,59,Private,HS-grad,9,Divorced,Tech-support,White,Female,0,0,40,United-States,<=85k
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32540,45,State-gov,HS-grad,9,Separated,Adm-clerical,White,Female,0,0,40,United-States,<=85k
32543,45,Local-gov,Assoc-acdm,12,Divorced,Prof-specialty,White,Female,0,0,48,United-States,<=85k
32544,31,Private,Masters,14,Divorced,Other-service,Other,Female,0,0,30,United-States,<=85k
32546,37,Private,Assoc-acdm,12,Divorced,Tech-support,White,Female,0,0,40,United-States,<=85k


In [32]:
# Check if filtering worked
print(df[not_married].marital_status.value_counts())
print('')
print(df[married].marital_status.value_counts())

Divorced                 4206
Separated                 937
Married-spouse-absent     369
Name: marital_status, dtype: int64

Married-civ-spouse    14022
Name: marital_status, dtype: int64


In [43]:
# Binary encode categorical target variable
df.loc[df.marital_status.isin(neg_marital_status), 'marital_status'] = 0
df.loc[df.marital_status == 'Married-civ-spouse', 'marital_status'] = 1

In [44]:
df.marital_status.value_counts()

1    14022
0     5512
Name: marital_status, dtype: int64

## 3. Null Accuracy

#### This is a good time to acknowledge null accuracy. If we assume every marraige will be successful, we'd be correct 71.7% of the time.

In [49]:
num_married = df.marital_status.value_counts()[1]
num_divorced = df.marital_status.value_counts()[0]

null_accuracy = num_married / (num_married + num_divorced)
null_accuracy

0.7178253301935088

## Move cleaned dataset to a new workbook

In [92]:
df.to_csv('marriage.csv')